In [387]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import math
from sklearn.preprocessing import StandardScaler


In [388]:
path = r"data_prueba/prueba_naive_bayes.xlsx"
data = pd.read_excel(path)
data

,Diario,Autor,Fecha,Título,Texto,Vínculo,Clase
0,El Espectador,Nicolás Rodríguez,16 de febrero de 2019,"De Rubio a Acevedo, pasando por Duque",Al presidente Donald Trump poco o nada le inte...,https://web.archive.org/web/20190216230452/htt...,Política
1,El Espectador,Salomón Kalmanovitz,15 de septiembre de 2019,Recentralización,Una de las facetas que acompañan el autoritari...,https://web.archive.org/web/20190916062046/htt...,Política
2,El Espectador,Adriana Cooper,17 de octubre de 2019,La primera alcaldesa de Medellín,Cuando uno revisa los libros sobre historia de...,https://web.archive.org/web/20191018032148/htt...,Política
3,El Espectador,Margarita Flórez,16 de febrero de 2019,"Fracking, luz amarilla a temas de participació...",Basándonos en la presentación realizada esta s...,https://web.archive.org/web/20190217013801/htt...,Medio Ambiente
4,El Espectador,Indalecio Dangond B.,23 de febrero de 2019,La hora del biodiésel,"Primero fue Medellín, ahora le tocó el turno a...",https://web.archive.org/web/20190223102037/htt...,Medio Ambiente
5,El Espectador,Julián López de Mesa Samudio,6 de marzo de 2019,"Biofilia, árboles y ciudad","En 1984, el gran entomólogo, sociobiólogo y do...",https://web.archive.org/web/20190307141807/htt...,Medio Ambiente
6,El Espectador,Santiago Montenegro,11 de febrero de 2019,“Roma”,"Guillermo del Toro, Alejandro González Iñárrit...",https://web.archive.org/web/20190212015339/htt...,Cultura
7,El Espectador,Juan Carlos Botero,14 de junio de 2019,Arte que dura minutos,"La Bienal de Venecia concluyó hace poco, y el ...",https://web.archive.org/web/20190615034748/htt...,Cultura
8,El Espectador,Juan Carlos Gómez,21 de julio de 2019,El futuro de Netflix,El lanzamiento de la tercera temporada de La c...,https://web.archive.org/web/20190722113712/htt...,Cultura
9,El Espectador,Héctor Abad Faciolince,18 de agosto de 2019,Los niños de la guerra,"Lo que más me gusta de esta película, Monos, d...",https://web.archive.org/web/20190818130521/htt...,NaN


In [389]:
# Lista de palabras relacionadas con política
politica = [
    "democracia", "gobierno", "elecciones",  "corrupción",
    "izquierda", "derecha", "populismo", "liberalismo", "conservadurismo",
    "políticas", "partidos", "voto", "legislación", "oposición",
    "presidente", "congreso", "liderazgo", "populista", "reforma"
]

# Lista de palabras relacionadas con medio ambiente
medio_ambiente = [
    "sostenibilidad", "climático", "contaminación", "biodiversidad", "renovables",
    "deforestación", "reciclaje", "emisiones", "ecología", "conservación",
    "calentamiento", "ambiental", "polución", "agotamiento", "residuos",
    "ecológica", "carbono", "protección ambiental", "ecosistema", "sostenible"
]

# Lista de palabras relacionadas con cultura
cultura = [
    "arte", "literatura", "cine", "música", "teatro",
    "patrimonio", "tradición", "identidad", "diversidad", "folclore",
    "creatividad", "exposición", "festival", "crítica", "vanguardia",
     "artística", "movimiento", "cultural", "intercultural", "película"
]


# Diccionario 
palabras_clave = {
    "politica": politica,
    "medio_ambiente": medio_ambiente,
    "cultura": cultura
}


# Pasar texto a las features de entrada

In [390]:
# Función para convertir texto a vector de características
def texto_a_vector(texto):
    # Contar palabras de cada categoría
    palabras = texto.lower().split()
    politica_count = sum(1 for palabra in palabras if palabra in politica)
    ambiente_count = sum(1 for palabra in palabras if palabra in medio_ambiente)
    cultura_count = sum(1 for palabra in palabras if palabra in cultura)
    return np.array([politica_count, ambiente_count, cultura_count])

# Funcion softmax

In [391]:
# Función softmax
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Softmax normal 
def softmax_simple(z):
    exp_z = np.exp(z)
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Función de pérdida

In [392]:
# Función de pérdida (entropía cruzada)
def cross_entropy(y_true, y_pred):
    m = y_true.shape[0]
    log_likelihood = -np.log(y_pred[range(m), y_true])
    return np.sum(log_likelihood) / m

## Gradiente estocástico

In [393]:
def gradiente_estocastico_descendente(X_train, y_train, learning_rate=0.01, iterations=10, softmax_func=softmax_simple):

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)

    n_samples, n_features = X_train.shape
    n_classes = len(np.unique(y_train))
    
    # Inicializar pesos y bias pequeños
    weights = np.random.randn(n_classes, n_features) * 0.01
    bias = np.zeros(n_classes)
    
    # Entrenamiento con gradiente estocástico
    for iter in range(iterations):
        # Mezclar los datos en cada época
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        g_W = []
        for i in indices:
            # Seleccionar un ejemplo aleatorio
            x = X_scaled[i:i+1]  # Mantener como matriz 2D (1, n_features)
            y = y_train[i]
            
            # Calcular logits
            z = np.dot(x, weights.T) + bias
            
            # Calcular probabilidades con softmax
            probs = softmax_func(z)
            
            # Crear vector one-hot de la clase verdadera
            one_hot = np.zeros(n_classes)
            one_hot[y] = 1
            
            # Calcular gradientes
            error = probs - one_hot
            grad_weights = np.dot(error.T, x)
            grad_bias = error.sum(axis=0)
            
                                    
            # Actualizar pesos y bias
            weights -= learning_rate * grad_weights
            bias -= learning_rate * grad_bias

        print(f"Iteración {iter+1}")
        print(np.linalg.norm(grad_weights))
        g_W.append(np.linalg.norm(grad_weights))
            
    return weights, bias, scaler, g_W

In [394]:
def predecir(X, weights, bias, scaler):
    X_scaled = scaler.transform(X)
    z = np.dot(X_scaled, weights.T) + bias
    return np.argmax(softmax_simple(z), axis=1)

In [395]:
# Documentos y clases para entrenamiento y prueba
documents = data["Texto"].tolist()[:9]  # 9 documentos para entrenamiento
classes = data["Clase"].tolist()[:9]    # clases correspondientes
doc_test = data["Texto"][9]             # documento para prueba

In [396]:
clases_unicas = list(set(classes))  
clase_a_numero = {clase: idx for idx, clase in enumerate(clases_unicas)}
y_train = np.array([clase_a_numero[clase] for clase in classes])

In [397]:
clase_a_numero

{'Medio Ambiente': 0, 'Política': 1, 'Cultura': 2}

In [398]:
y_train

array([1, 1, 1, 0, 0, 0, 2, 2, 2])

In [399]:
X_train = np.array([texto_a_vector(doc) for doc in documents])

In [400]:
X_train

array([[3, 0, 0],
       [8, 0, 0],
       [7, 0, 0],
       [2, 3, 0],
       [2, 5, 1],
       [1, 6, 0],
       [1, 0, 3],
       [0, 1, 5],
       [0, 0, 0]])

In [401]:
X_test = np.array([texto_a_vector(doc_test)])

In [402]:
weights, bias, scaler, g_b = gradiente_estocastico_descendente(X_train, y_train, learning_rate=0.01, iterations=100, softmax_func=softmax)

Iteración 1
1.097977443749785
Iteración 2
0.6887088965339859
Iteración 3
1.5039045231197192
Iteración 4
1.4281541106629967
Iteración 5
0.7239838522929607
Iteración 6
1.276050192661326
Iteración 7
0.6362147916651009
Iteración 8
0.9496335349946741
Iteración 9
1.2632364668970637
Iteración 10
0.679892176778557
Iteración 11
0.8594475860734607
Iteración 12
0.9173054836231026
Iteración 13
0.5832722248047538
Iteración 14
0.6515382116328079
Iteración 15
0.7590628544990441
Iteración 16
0.7276878368546946
Iteración 17
0.6983327076777806
Iteración 18
0.6279340239730966
Iteración 19
0.735126084163867
Iteración 20
0.6572554076516773
Iteración 21
0.5707387488981097
Iteración 22
0.5437476223521875
Iteración 23
0.515996736478643
Iteración 24
0.5103655699044304
Iteración 25
0.5163413981711618
Iteración 26
0.5502586504699571
Iteración 27
0.43145008120379824
Iteración 28
0.5832835471338564
Iteración 29
0.4039924771279325
Iteración 30
0.43771975751082565
Iteración 31
0.5726904942929238
Iteración 32
0.48824

In [403]:
bias

array([ 0.02749125, -0.03280917,  0.00531792])

In [404]:
clase_indice = predecir(X_test, weights, bias, scaler)[0]
nombre_clase = clases_unicas[clase_indice]
nombre_clase

'Cultura'